In [1]:
import os
import keras
from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator


from keras.models import Sequential
# 卷積層、池化層
from keras.layers import Conv2D, MaxPooling2D
# 神經網絡正規化
from keras.regularizers import l1, l2, l1_l2 
# 神經網絡隨機拋棄、批次標準化、激活函數、平坦層
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten 
# 優化器(Optimizer)
from keras.optimizers import RMSprop, Adam
# Early Stopping
from keras.callbacks import EarlyStopping
# 自動存取目前模型的最佳權重
from keras.callbacks import ModelCheckpoint
# Reduce Learning Rate
from keras.callbacks import ReduceLROnPlateau
# 訓練模型時，利用 callback 紀錄訓練過程
from keras.callbacks import Callback
# 自訂損失函數
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline


Using TensorFlow backend.


##讀取資料

In [2]:
# 讀取資料並檢視
( x_train, y_train ), ( x_test, y_test ) = cifar10.load_data( )
print( 'x_train shape:', x_train.shape )
print( x_train.shape[0], 'train samples' )
print( x_test.shape[0], 'test samples' )

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# 設定超參數

In [0]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

'''調整學習率'''
lr = 0.0001  # 預設為 0.001

## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32 x 32 x 3) = (50000, 3072)

### 資料前處理 - 平坦化、標準化 及 獨熱編碼(OneHot Encoding)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape( 50000, 3072 ) 
x_test = x_test.reshape( 10000, 3072 )

# 將資料變為 float32 並標準化
x_train = x_train.astype( 'float32' )
x_test = x_test.astype( 'float32' )
x_train /= 255
x_test /= 255

print( x_train.shape[0], 'train samples' )
print( x_test.shape[0], 'test samples' )


# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical( y_train, num_classes )
y_test = keras.utils.to_categorical( y_test, num_classes )

50000 train samples
10000 test samples


### 建立神經網絡模型

In [5]:
model = Sequential()
model.add( Dense( 512, activation = 'relu', input_shape = ( 3072, ) ) )
model.add( Dropout( 0.2 ) )
model.add( Dense( 512, activation = 'relu' ) )
model.add( Dropout( 0.2 ) )
model.add( Dense( num_classes, activation = 'softmax' ) )

model.summary( )

model.compile( loss = 'categorical_crossentropy',
                      optimizer = RMSprop( lr = lr ),
                      metrics = [ 'accuracy' ] )

history = model.fit( x_train, y_train,
                           batch_size = batch_size,
                           epochs = epochs,
                           verbose = 1,
                           validation_data = ( x_test, y_test ) 
                           )

score = model.evaluate( x_test, y_test, verbose = 0 )

print( 'Test loss:', score[0] )
print( 'Test accuracy:', score[1] )

W0805 08:13:38.530593 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 08:13:38.572847 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 08:13:38.580179 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 08:13:38.604388 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0805 08:13:38.617275 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________


W0805 08:13:38.714424 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0805 08:13:38.861399 140574694369152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 23s 460us/step - loss: 1.9535 - acc: 0.2968 - val_loss: 1.7792 - val_acc: 0.3695
Epoch 2/10
50000/50000 [==============================] - 23s 454us/step - loss: 1.7672 - acc: 0.3691 - val_loss: 1.6969 - val_acc: 0.3929
Epoch 3/10
50000/50000 [==============================] - 22s 439us/step - loss: 1.6915 - acc: 0.3980 - val_loss: 1.6073 - val_acc: 0.4340
Epoch 4/10
50000/50000 [==============================] - 22s 439us/step - loss: 1.6424 - acc: 0.4176 - val_loss: 1.6478 - val_acc: 0.4122
Epoch 5/10
50000/50000 [==============================] - 22s 434us/step - loss: 1.5960 - acc: 0.4360 - val_loss: 1.5837 - val_acc: 0.4324
Epoch 6/10
50000/50000 [==============================] - 22s 440us/step - loss: 1.5627 - acc: 0.4486 - val_loss: 1.5430 - val_acc: 0.4487
Epoch 7/10
50000/50000 [==============================] - 22s 442us/step - loss: 1.5336 - acc: 0.4549 - val_loss: 1.

# 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [6]:
( x_train, y_train ), ( x_test, y_test ) = cifar10.load_data( )
print( 'x_train shape:', x_train.shape )
print( x_train.shape[0], 'train samples' )
print( x_test.shape[0], 'test samples' )
x_train = x_train.astype( 'float32' )
x_test = x_test.astype( 'float32' )
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical( y_train, num_classes )
y_test = keras.utils.to_categorical( y_test, num_classes )

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


##建立卷積神經網絡模型

In [7]:
model = Sequential( )
model.add( Conv2D( 32, ( 3, 3 ), 
                              padding = 'same',
                              input_shape = x_train.shape[ 1: ] 
                              ) )

model.add( Activation( 'relu' ) )
model.add( Conv2D( 32, ( 3, 3 ) ) )
model.add( Activation( 'relu' ) )
model.add( MaxPooling2D( pool_size = ( 2, 2 ) ) )
model.add( Dropout( 0.25 ) )

model.add( Conv2D( 64, ( 3, 3 ), 
                              padding = 'same' 
                              ) )
model.add( Activation( 'relu' ) )
model.add( Conv2D( 64, ( 3, 3 ) ) )
model.add( Activation( 'relu' ) )
model.add( MaxPooling2D( pool_size = ( 2, 2 ) ) )
model.add( Dropout( 0.25 ) )

model.add( Flatten( ) )
model.add( Dense( 512 ) )
model.add( Activation( 'relu' ) )
model.add( Dropout( 0.5 ) )
model.add( Dense( num_classes ) )
model.add( Activation( 'softmax' ) )
model.summary( )

model.compile( loss = 'categorical_crossentropy',
                      optimizer = RMSprop( lr = lr ),
                      metrics = [ 'accuracy' ] 
                      )

history = model.fit( x_train, y_train,
                           batch_size = batch_size,
                           epochs = epochs,
                           verbose = 1,
                           validation_data = ( x_test, y_test ) 
                           )

score = model.evaluate( x_test, y_test, verbose = 0 )

print( 'Test loss: ', score[0] )
print( 'Test accuracy: ', score[1] )

W0805 08:17:24.095393 140574694369152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?  
  Ans : 將學習率調小( 0.001 --> 0.0001 )，CNN的學習明顯緩慢，損失函數從 0.73565 升至 1.0946，且準確率也較原本預設的結果低。  
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?  
  Ans : DNN ; CNN 有透過池化層篩選特徵及降低資料維度。